In [ ]:
from time import time
import numpy as np
import matplotlib.pyplot as pl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
import pandas
import math
import sys
import subprocess
from sklearn import metrics
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from pylab import plot,show
from numpy import vstack,array
from numpy.random import rand
from scipy.cluster.vq import vq

class Point:
    def __init__(self, coords):
        self.coords = coords
        self.n = len(coords)
    def __repr__(self):
        return str(self.coords)
        
def getDistance(a, b):    
    ret = reduce(lambda x,y: x + pow((a.coords[y]-b.coords[y]), 2),range(a.n),0.0)
    return math.sqrt(ret)
        
        
class Cluster:
    def __init__(self, points):
       
        self.points = points
        
        self.n = points[0].n
        
        for p in points:
            if p.n != self.n: raise Exception("Incorrect dimension")
            
        self.centroid = self.calculateCentroid()
    
    def update(self, points):
        old_centroid = self.centroid
        self.points = points
        self.centroid = self.calculateCentroid()
    
    def calculateCentroid(self):
        numPoints = len(self.points)
        coords = [p.coords for p in self.points]
        unzipped = zip(*coords)
        centroid_coords = [math.fsum(dList)/numPoints for dList in unzipped]
        
        return Point(centroid_coords)

def makeDataPoint(i):
    p = Point(i)
    return p

def myKMeans(points, k, a):
    random.seed(a)
    initial = random.sample(points, k)    
    clusters = [Cluster([p]) for p in initial]
    loopCounter = 0
    lists = [ [] for c in clusters]
    clusterCount = len(clusters)
    loopCounter += 1
    
    for p in points:
        smallest_distance = getDistance(p, clusters[0].centroid)    
        clusterIndex = 0    
        for i in range(clusterCount - 1):
            distance = getDistance(p, clusters[i+1].centroid)
            if distance < smallest_distance:
                smallest_distance = distance
                clusterIndex = i+1            
        lists[clusterIndex].append(p)
        biggest_shift = 0.0
    for i in range(clusterCount):
        clusters[i].update(lists[i])   
    return clusters

def claculateDestortion(clustersCalculated):
    destortion=0.0
    for i in clustersCalculated:
        for j in i.points:
            destortion=destortion+getDistance(j, i.centroid) 
    return destortion
            
def myKMeansplusplus(k,noOfIterations,points):
    #clustersCalculated=[[]for i in  noOfIterations]
    clustersCalculated = [ [] for i in range (1,noOfIterations)]
    a = 10102014
    for i in range(0,noOfIterations-1):
        clustersCalculated[i]=myKMeans(points, k, a)
        a+=10000
        #print(clustersCalculated[i][0].centroid.coords)
        Initialclusters=clustersCalculated[i]
        destortion=claculateDestortion(clustersCalculated[i])
        print('Iteration')
        print(i)
        print('Destortion')
        print(destortion)
    
#np.random.seed(11112014)
digits = load_iris()
k=3
data = scale(digits.data)
data = PCA(n_components=2).fit_transform(data)
points = [makeDataPoint(i) for i in data]
noOfIterations=99
a=10102014
clusters = myKMeans(points, k, a)
iterativeCluster = myKMeansplusplus(k,noOfIterations,points)
plt.show()
centr=[clust.centroid.coords for clust in clusters]
clusterpoints=[po.coords for po in clusters[0].points]
clusterpoints2=[po.coords for po in clusters[1].points]
clusterpoints3=[po.coords for po in clusters[2].points]
#clusterpoints4=[po.coords for po in clusters[3].points]
#clusterpoints5=[po.coords for po in clusters[4].points]
#clusterpoints6=[po.coords for po in clusters[5].points]
#clusterpoints7=[po.coords for po in clusters[6].points]

pl.scatter(*zip(*centr),marker='x',s=150,c='r')
pl.scatter(*zip(*clusterpoints),c='g')
pl.scatter(*zip(*clusterpoints2),c='b')
pl.scatter(*zip(*clusterpoints3),c='k')
#pl.scatter(*zip(*clusterpoints4),c='w')
#pl.scatter(*zip(*clusterpoints5),c='y')
#pl.scatter(*zip(*clusterpoints6),c='c')
#pl.scatter(*zip(*clusterpoints7),c='m')
pl.show()

In [8]:
from time import time
import numpy as np
import pylab as pl

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

np.random.seed(11112014)

digits = load_iris()
data = scale(digits.data)

n_samples, n_features = data.shape
n_digits = len(np.unique(digits.target))
labels = digits.target

sample_size = 300

print("digits: %d, \t samples %d, \t features %d"
      % (n_digits, n_samples, n_features))


def k_means(estimator, name, data):
    t0 = time()
    estimator.fit(data)

k_means(KMeans(init='k-means++', n_clusters=n_digits, n_init=3),name="k-means++", data=data)
pca = PCA(n_components=n_digits).fit(data)
reduced_data = PCA(n_components=2).fit_transform(data)
kmeans = KMeans(init='k-means++', n_clusters=n_digits, n_init=3)
kmeans.fit(reduced_data)
h = .01
x_min, x_max = reduced_data[:, 0].min() + 1, reduced_data[:, 0].max() - 1
y_min, y_max = reduced_data[:, 1].min() + 1, reduced_data[:, 1].max() - 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
pl.figure(1)
pl.clf()
pl.imshow(Z, interpolation='nearest',extent=(xx.min(), xx.max(), yy.min(), yy.max()),cmap=pl.cm.Paired,aspect='auto', origin='lower')
pl.plot(reduced_data[:, 0], reduced_data[:, 1], 'k.', markersize=2)
centroids = kmeans.cluster_centers_
pl.scatter(centroids[:, 0], centroids[:, 1],marker='x', s=169, linewidths=3,color='w', zorder=10)
pl.title('Using K means cluster')
pl.xlim(x_min, x_max)
pl.ylim(y_min, y_max)
pl.xticks(())
pl.yticks(())
pl.show()

digits: 3, 	 samples 150, 	 features 4
